In [2]:
import os
import numpy as np
import spatialmath.base as smb
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import mpl_toolkits.mplot3d.art3d as art3d
import matplotlib
from roboticstoolbox.robot.ERobot import ERobot
from spatialgeometry import Cuboid
from spatialmath import SE3
from matplotlib.patches import Rectangle
from matplotlib.colors import ListedColormap
from bresenham import bresenham
from tqdm import tqdm
matplotlib.use('Qt5Agg')

In [3]:
class TwoLink(ERobot):
    '''
    Initialize a 2link robot
    '''
    def __init__(self):
        links, name, urdf_string, urdf_filepath = self.URDF_read("two_link.urdf", tld=os.getcwd())
        super().__init__(
            links,
            name=name,
            manufacturer="ARClab",
            urdf_string=urdf_string,
            urdf_filepath=urdf_filepath,
        )
        
        self.qr = np.array([0, 0.0])
        self.qz = np.zeros(2)

        self.addconfiguration("qr", self.qr)
        self.addconfiguration("qz", self.qz)

In [4]:
def add_rectangle (xy , width , height , ax, _color):
    '''
    Add a rectangle patch to the environment .
    : param xy: xy corner of the left - bottom corner .
    : param height : height of the window .
    : param width : width of the window .
    : param ax: the axis on which to plot .
    '''
    obj_patch = Rectangle (xy , width , height, color = _color)
    ax.add_patch ( obj_patch )
    art3d.pathpatch_2d_to_3d ( obj_patch , z=0, zdir = 'z')

In [5]:
robot = TwoLink()
# robot.plot([np.pi/4,np.pi/2], backend = 'pyplot')

In [6]:
# Create cuboids for obstacles in the environment
box1 = Cuboid ( scale =[ 1 , 30 , 1 ], pose = SE3(-11.5, 20, 0), collision = True )    # Top-left quadrant
box2 = Cuboid ( scale =[ 1 , 35 , 1 ], pose = SE3(11.5, 17.5, 0), collision = True )   # Top-right quadrant
box3 = Cuboid ( scale =[ 1 , 35 , 1 ], pose = SE3(-11.5, -17.5, 0), collision = True ) # Bottom-left quadrant
box4 = Cuboid ( scale =[ 1 , 30 , 1 ], pose = SE3(11.5, -20, 0), collision = True )    # Bottom-right quadrant

In [7]:
# Create an empty figure
fig = plt.figure()

q1 = [0,0]

# Plot the robot pose
plot_fig = robot.plot(q=q1, fig=fig, backend = 'pyplot')

# Set plot view to the X-Y axis.
plot_fig.ax.view_init(90, -90, 0)

# Add a rectangle with bottom left corner at xy location and of size heigh and width
# Add rectangles to visualize obstacles in the environment
add_rectangle([-12, 5], 1, 30, plot_fig.ax, 'r')   # Top-left quadrant
add_rectangle([11, 0], 1, 35, plot_fig.ax, 'g')    # Top-right quadrant
add_rectangle([-12, -35], 1, 35, plot_fig.ax, 'b') # Bottom-left quadrant
add_rectangle([11, -35], 1, 30, plot_fig.ax, 'yellow')  # Bottom-right quadrant

# Set axis limits
plot_fig.ax.set_xlim(-40, 40)
plot_fig.ax.set_ylim(-35, 35)

plt.rcParams['figure.figsize'] = [24, 10]

plt.show()

![alt text](env_with_obstacles.jpg "Title")

### Configuration Space of the Robot

In [7]:
# Limits for both the robot links
min_q_link1 = 0
max_q_link1 = 360
min_q_link2 = round(-0.97 * 180)
max_q_link2 = round(0.97 * 180)

# Initialize configuration space matrix
conf_space = np.zeros((350, 360))

# Loop to check collisions with obstacles and plot configuration space
for q1 in range(min_q_link1, max_q_link1):
    for q2 in range(min_q_link2, max_q_link2 + 1):
        
        # Convert angles to radians
        temp_q1 = np.pi * q1 / 180
        temp_q2 = np.pi * q2 / 180
        
        q  = [temp_q1, temp_q2]
        q2 += 175
        
        # Check collisions with each obstacle
        if robot.iscollided(q, box1):
            conf_space[q2, q1] = 50
        
        elif robot.iscollided(q, box2):
            conf_space[q2, q1] = 100
        
        elif robot.iscollided(q, box3):
            conf_space[q2, q1] = 150
        
        elif robot.iscollided(q, box4):
            conf_space[q2, q1] = 200

In [8]:
# Plot Configuration space
cmap = ListedColormap(['black', 'r', 'g', 'b', 'yellow'])
plt.xlabel('Link 1 angles (in degrees)')
plt.ylabel('Link 2 angles (in degrees)')
plt.title('Configuration Space of 2-link 2D Planar Robot')
plt.rcParams['figure.figsize'] = [20, 8]
plt.imshow(conf_space, cmap = cmap, origin='lower', extent=[-180, 180, -175, 175])

# Transpose of conf_space to make it compatible with [q1, q2]
conf_space     = conf_space.T

![alt text](conf_space.jpg "Title")

### RRT* Implementation

In [9]:
def SAMPLEFREE(_conf_space_lim):
    
    # Input:
    # _conf_space_lim: limits of configuration space in the order min_q_link1, max_q_link1, min_q_link2, max_q_link2
    
    # Output:
    # [_q1, _q2]: randomly selected configuration from the configuration space (list)
    
    _q1 = np.random.randint(_conf_space_lim[0], _conf_space_lim[1])
    _q2 = np.random.randint(_conf_space_lim[2], _conf_space_lim[3])
    return [_q1, _q2]

def NEAREST(_V, _E, _q_rand):
    
    # Input:
    # _V: list of tree vertices
    # _E: list of tree edges
    # _q_rand: randomly selected configuration from the configuration space
    
    # Output:
    # q_nearest: vertex which is nearest to _q_rand (list)
    
    temp_arr1     = np.asarray(_V)
    
    if len(_E) != 0:
        for _tuple in _E:
            if list(_tuple[0]) not in _V:
                temp_arr1 = np.vstack((temp_arr1, _tuple[0]))
            
            if list(_tuple[1]) not in _V:
                temp_arr1 = np.vstack((temp_arr1, _tuple[1]))        
    
    temp_arr2     = temp_arr1 - np.asarray(_q_rand)
    temp_arr_norm = np.linalg.norm(temp_arr2, axis = 1)
    index         = np.argmin(temp_arr_norm)
    _q_nearest    = temp_arr1[index]
    
    return list(_q_nearest)

def STEER(_q_rand, _q_nearest, _eps):
    
    # Input:
    # _q_rand: randomly selected configuration from the configuration space
    # _q_nearest: vertex which is nearest to _q_rand
    # _eps: constant epsilon
    
    # Output:
    # _q_new: New point which is _eps distance away from _q_nearest (list)
    
    # Check to ensure non-zero norm
    if np.all(np.asarray(_q_rand) == np.asarray(_q_nearest)):
        _q_new = _q_nearest
    
    else:
        q_bar  = np.asarray(_q_rand) - np.asarray(_q_nearest)
        q_bar  = q_bar / np.linalg.norm(q_bar)
        _q_new = np.asarray(_q_nearest) + (_eps * q_bar)
        _q_new = _q_new.round().astype(int)
        
        if _q_new[0] > 359:
            _q_new[0] = 359
        
        elif _q_new[0] < 0:
            _q_new[0] = 0
        
        elif _q_new[1] > 349:
            _q_new[1] = 349
        
        elif _q_new[1] < 0:
            _q_new[1] = 0
    
    return list(_q_new)

def COLLISIONFREE(_q_nearest, _q_new, _conf_space):
    
    # Input:
    # _q_nearest: vertex which is nearest to _q_rand
    # _q_new: New point which is _eps distance away from _q_nearest
    
    # Output:
    # bool: Return False if it is a collision in configuration space, otherwise True
    
    x0 = _q_nearest[0]
    y0 = _q_nearest[1]
    x1 = _q_new[0]
    y1 = _q_new[1]
    
    line_pts = list(bresenham(x0, y0, x1, y1))
    
    for pt in line_pts:
        if _conf_space[pt] != 0:
            return False
    
    return True    

def NEAR(_V, _E, _q_new, _r):
    
    # Input:
    # _V: list of tree vertices
    # _E: list of tree edges
    # _q_new: New point which is _eps distance away from _q_nearest
    # _r: radius of the circle centered at _q_new
    
    # Output:
    # _q_near: list of nodes which fall inside the circle centered at _q_new (list)
    
    _q_near = []
    
    temp_arr     = np.asarray(_V)
    
    if len(_E) != 0:
        for _tuple in _E:
            temp_arr = np.vstack((temp_arr, _tuple[0]))
            temp_arr = np.vstack((temp_arr, _tuple[1]))   
 
    for pt in temp_arr:
        dist = np.linalg.norm(np.asarray(pt) - np.asarray(_q_new))
        if dist <= _r:
            if list(pt) not in _q_near:
                _q_near.append(list(pt))
    
    return _q_near

In [29]:
def motion_planner(_start_point, _goal_point):
    
    # Initialize V and E
    V              = [list(_start_point)]
    E              = []

    # Algorithm variables
    eps            = 10
    r_star         = 15
    num_iterations = 2000
    first_path_itr = 300

    # Initialize Cost Matrix
    cost_matrix    = np.ones((360, 350)) * np.inf
    cost_matrix[tuple(_start_point)] = 0

    # Initialize Parent dictionary
    parent_dict    = {}

    for i in tqdm(range(num_iterations)):
        q_rand    = SAMPLEFREE(conf_space_lim)
        q_nearest = NEAREST(V, E, q_rand)
        q_new     = STEER(q_rand, q_nearest, eps)
        if COLLISIONFREE(q_nearest, q_new, conf_space):
            q_near = NEAR(V, E, q_new, r_star)
            q_min  = q_nearest
            c_min  = cost_matrix[tuple(q_nearest)] + np.linalg.norm(np.asarray(q_nearest) - np.asarray(q_new))
            cost_matrix[tuple(q_new)] = c_min

            for _q in q_near:
                if COLLISIONFREE(_q, q_new, conf_space):
                    if cost_matrix[tuple(_q)] + np.linalg.norm(np.asarray(_q) - np.asarray(q_new)) < c_min:
                        q_min = _q
                        c_min = cost_matrix[tuple(_q)] + np.linalg.norm(np.asarray(_q) - np.asarray(q_new))                         

            V.append(q_new)
            E.append((q_min, q_new)) 
            if list(q_new) != q_min:
                parent_dict[tuple(q_new)] = q_min

            for _q in q_near:
                if _q != q_min and COLLISIONFREE(_q, q_new, conf_space):
                    if cost_matrix[tuple(q_new)] + np.linalg.norm(np.asarray(_q) - np.asarray(q_new)) < cost_matrix[tuple(_q)]:
                        if list(_q) != q_new:
                            parent_dict[tuple(_q)] = q_new
                        E.append((_q, q_new))
        
        # Store Tree and parent-dict for first-path
        if i == first_path_itr:
            V_prime           = V.copy()
            E_prime           = E.copy()
            parent_dict_prime = parent_dict.copy()
        
    # Get first and optimal path
    start = tuple(_start_point) 
    goal  = tuple(_goal_point) 
    c1    = tuple(NEAREST(V, [], list(goal))) 
    c2    = tuple(NEAREST(V_prime, [], list(goal)))

    # Optimal path initialization
    path1 = np.asarray(goal)
    path1 = np.vstack((path1, np.asarray(c1)))

    # First path initialization
    if COLLISIONFREE(goal, c2, conf_space):
        path2 = np.asarray(goal)
        path2 = np.vstack((path2, np.asarray(c2)))
    
    else:
        path2 = np.asarray(c2)
    
    print('Starting optimal path')
    
    # Optimal path
    while c1 != start:
        if c1 in parent_dict.keys():
            c1    = tuple(parent_dict[c1])
            path1 = np.vstack((path1, np.asarray(c1)))
    
    print('Optimal path found!')
    
    print('Starting first path')
    
    # First path
    while c2 != start:
        if c2 in parent_dict_prime.keys():
            c2 = tuple(parent_dict_prime[c2])
            path2 = np.vstack((path2, np.asarray(c2)))
    
    print('First path found!')
    
    return V, E, path1, path2

In [30]:
def plot_tree_path(_conf_space, _V, _E, _start, _goal, _path1, _path2):
    
    # Create a figure and an axis object
    fig, ax = plt.subplots()
    
    # Initialize an empty line object
    line, = ax.plot([], [])

    # Plot the configuration space
    plt.imshow(_conf_space.T, cmap = cmap, origin='lower', extent=[-180, 180, -175, 175])
    
    # Plot vertices of the tree
    for pt in _V:
        pt[0] = pt[0] - 180
        pt[1] = pt[1] - 175
        plt.scatter(pt[0], pt[1], color = 'r', s = 4)

    # Plot edges of the tree
    for _tuple in _E:
        a = _tuple[0][0] - 180
        b = _tuple[1][0] - 180
        c = _tuple[0][1] - 175
        d = _tuple[1][1] - 175
        
        plt.plot([a, b], [c, d], linewidth = 1, color = 'yellow')
    
    # Plot first and optimal paths
#     plt.plot(_path1[:, 0], _path1[:, 1], color = 'r', linewidth = 3)
    plt.plot(_path2[:, 0], _path2[:, 1], color = 'b', linewidth = 3)
    
    # Show start and goal points
    plt.scatter(_start[0], _start[1], color = 'g', s = 64)
    plt.scatter(_goal[0], _goal[1], color = 'white', s = 64)
    
    # Animation update function
    def update(frame):
        # Select points up to the current frame
        current_points = _path1[:frame+1]

        # Update the line data
        line.set_data(current_points[:, 0], current_points[:, 1])
        
        line.set_color('r')
        line.set_linewidth(4)

        return line,

    # Create the animation
    animation = matplotlib.animation.FuncAnimation(fig, update, frames=len(_path1), interval=100, blit=True)

    # Save the animation as a GIF
    animation.save('c_space1.gif', writer='pillow')
    
    # Show the plot
    plt.show()

In [31]:
# Limits for both the robot links
min_q_link1      = 0 #-180
max_q_link1      = 360 #180
min_q_link2      = round(-0.97 * 180) + 175
max_q_link2      = round(0.97 * 180) + 175
 
conf_space_lim   = [min_q_link1, max_q_link1, min_q_link2, max_q_link2]

# Define waypoints
x1        = [0, 20, 0]
x2        = [0, -20, 0]
x3        = [-19.43, 4.62, 0]
x4        = [2, 2, 0]
x5        = [-8, 0, 0]
x6        = [-5, -5, 0]
x7        = [-9, 0, 0]

waypoints = []
waypoints.append(x1)
waypoints.append(x2)
waypoints.append(x3)
waypoints.append(x4)
waypoints.append(x5)
waypoints.append(x6)
waypoints.append(x7)

# Get robot configurations for waypoints
waypoint_conf  = [[0,0]]

for waypoint in waypoints:
    conf  = robot.ikine_LM(SE3(waypoint), waypoint_conf[-1])[0]
    waypoint_conf.append(conf)

waypoint_conf.remove([0,0])

# Convert the waypoint configurations as per env limits in degrees
waypoint_conf  = (np.asarray(waypoint_conf) * 180 / np.pi) + np.array([180, 175])
waypoint_conf  = waypoint_conf.round().astype(int)

# Call motion planner to get first path and optimal path
# path1: optimal path
# path2: first path

start_pt           = waypoint_conf[5]
goal_pt            = waypoint_conf[6]
V, E, path1, path2 = motion_planner(start_pt, goal_pt)


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:31<00:00, 21.81it/s]

Starting optimal path
Optimal path found!
Starting first path
First path found!


In [32]:
# Bring all the variables in oroginal q1 and q2 limits
# q1 (- [-180, 180]
# q2 (- [-175, 175]

path1    = path1 - np.array([180, 175])
path1    = np.flip(path1, 0)
path2    = path2 - np.array([180, 175])
start_pt = start_pt - np.array([180, 175])
goal_pt  = goal_pt - np.array([180, 175])

# Call the plotting function to plot paths and obtain GIF in configuration space
plot_tree_path(conf_space, np.copy(V), np.copy(E), start_pt, goal_pt, path1, path2)

In [33]:
# Create GIF of the robot in cartesian space with obstacles

# Convert angles to radians
joint_angles = (path1) * np.pi / 180
i = 1

for angle in joint_angles:
    fig = plt.figure()

    q1 = angle

    # Plot the robot pose
    plot_fig = robot.plot(q=q1, fig=fig, backend = 'pyplot')

    # Set plot view to the X-Y axis.
    plot_fig.ax.view_init(90, -90, 0)

    # Add a rectangle with bottom left corner at xy location and of size heigh and width
    # Add rectangles to visualize obstacles in the environment
    add_rectangle([-12, 5], 1, 30, plot_fig.ax, 'r')   # Top-left quadrant
    add_rectangle([11, 0], 1, 35, plot_fig.ax, 'g')    # Top-right quadrant
    add_rectangle([-12, -35], 1, 35, plot_fig.ax, 'b') # Bottom-left quadrant
    add_rectangle([11, -35], 1, 30, plot_fig.ax, 'yellow')  # Bottom-right quadrant

    # Set axis limits
    plot_fig.ax.set_xlim(-40, 40)
    plot_fig.ax.set_ylim(-35, 35)

#     plt.show()

    plt.savefig(str(i) + '.png')
    plt.close(fig)
    
    i += 1